# A new Pipeline Approach: CORRAL

<center><img src='./images/3.jpg' width=360 Height=240></center>

## The TOROS Pipeline's Engine

### Co-workers: J. B. Cabral, M. Beroiz, TOROS Collaboration

## What is a pipeline?

*A pipeline can be understood in a collection of filters and connectors that consume data to perform a transformation of it on a linear and repetitive fashion.*

This data that is consumed can be named as a *stream* and each filter is a transformation. Connectors between the filters are the schema of the pipeline, and it inspires a *always forward* structure.

<center><img src='./images/pipeline.png' align='middle'></center>

####Some Pipelines

* SDSS
* Pan STARSS

## Why do we do this?


* We need this for TOROS

* Observational astronomers are facing a data *tsunami*, and synoptic surveys are one of the more data intensive tasks today.

* Telescopes of any size can produce massive amounts of data, and TOROS needs real-time processing

* Processing is time consuming for humans, and most of the tasks involved are likely to fail.

* Handling meta-data is key to a Sinoptic Survey 

## The TORITOS Project

* The first observation campaign needs.

    * Where to host data
    * A data base to record process
    * Computational power (real-time analysis)
    * Automatization of every process
    * A report system (lack of connectivity)
    * Error handling mechanisms
    
<center><img src='./images/toritos_site.png' align='middle'></center>

## CORRAL
<img src='./images/9.jpg' width=120 height=60>

**Corral is a framework designed to create fully integrated pipelines without tears and headaches.**

Provides a structure where your pipeline metadata can be stored, mined, and analysed.


It makes use of some computational formalisms and tools:
* MVC pattern
* ETL operation routines
* OOP (Python native)
* SQL database fully integrated
* Pipeline branching
* Asynchronus and embarrasing parallel processing


** Testing and Code quality measurements integrated.** 

## Toritos pipeline design

A main feature of this new Pipeline design approach is ***Model-View-Controller*** pattern.

The TORITOS pipeline needs three central entities:

* The **Models**

* The **Loader** and **Steps**  (as *controllers*)

* The **Alerts** (as user *views*)

### The models

These are the designed data structures that will be interacting with the processing steps, and are the storage of information that the pipeline handles.

It is correct to say that are the SQL data tables definitions which the pipeline works with. 

Here I should import corral db to comply with model requirements

In [5]:
from sqlalchemy.orm import *
from sqlalchemy import *
from sqlalchemy.ext import declarative

Model = declarative.declarative_base(name="Model")

In [6]:
class Observatory(Model):
    """Model for observatories. SQLAlchemy Model object.
    """

    __tablename__ = 'Observatory'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True)
    latitude = Column(Float, nullable=False)
    longitude = Column(Float, nullable=False)
    description = Column(Text, nullable=True)

    def __repr__(self):
        return self.name


In [7]:
class Campaign(Model):

    __tablename__ = 'Campaign'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True)
    description = Column(Text, nullable=True)

    observatory_id = Column(Integer, ForeignKey('Observatory.id'))
    observatory = relationship(
        "Observatory", backref=backref('campaigns', order_by=id))
    ccd_id = Column(Integer, ForeignKey('CCD.id'))
    ccd = relationship(
        "CCD", backref=backref('campaigns', order_by=id))

    def __repr__(self):
        return self.name


<center><img src='./images/db.png'></center>

### The Steps

The processing operations performed during each stage of the pipeline are encapsulated in *atomic Steps*, i.e. each one is to be performed completely or failed fatally. 

They cannot have a mixed outcome, and every time a step process is triggered a database entry is recorded, to help mantain reproducibility of results and error tracking.


####Loader

The Loader is the very first step to be executed and is the only one that is able to input data on the stream of the pipeline. It needs to exist and it can be hosted inside a cronjob or similar task managers.

####General Steps and branching

The general step entity is formalised in **filters** and **connectors**.

* Filter: works by Quering the SQL database, extracting data, and transforming it
* Connector: works by loading the result of a filter operation into new database records.

This can be seen inside a *ETL* closed process

#### Extract Transform and Load

This is a **processing recipe** in which data is queried from a DB, some processing takes place, and after that a new record is registered into a DB (can be the same).

<center><img src='./images/ETL-Process.png'></center>

#### Branching

A line of data stream can be processed in parallel, if there is no concurrence between its partner processes.

<center><img src='./images/BranchingPipeline.png'></center>

### The Alerts

Whenever a previously defined condition is satisfied, or a preferred state is reached, an *alert* is triggered.

CORRAL can communicate these alerts by SMS or e-mail, or even web services (like a Django web app).

<center><img src='./images/notifications.png' width=360 height=360></center>

Inside CORRAL any alert can be defined, and any channel compatible with Python will communicate the information.

## Conclusion

This Pipeline framework has been applied for TORITOS campaign, and it will be fully deployed in next campaign.

* CORRAL is a framework ideal for research, and scientific DB hosting and data mining.
* It is able to work under many environment conditions, being really flexible
* It is completely written in Python, and TORITOS does not depend on any other language or software.
* We can guarantee code quality and reproducibility of results
* Provides a parallelization of the chain processes out of the box. 